In [ ]:
import pandas as pd
import numpy as np

# Define constants
num_employees = 10
days_in_year = 365
hours_per_day = 8  # Max working hours per day
max_days_off = 3  # Max consecutive days off
working_hours_per_week = 40  # Total working hours per week (5 days of 8 hours)
min_employees_working = int(np.ceil(num_employees * 0.75))  # At least 75% employees working

# Generate a list of employee names
employees = [f'Employee_{i+1}' for i in range(num_employees)]

# Generate all dates in 2025
dates = pd.date_range(start='2025-01-01', periods=days_in_year, freq='D')

# Create a DataFrame to hold the shifts for all employees
shifts = pd.DataFrame(index=dates)

# Initialize the work schedule for each employee
work_schedule = {employee: [] for employee in employees}
rest_schedule = {employee: [] for employee in employees}  # To track consecutive rest days

# Function to assign shifts
def assign_shifts():
    # Initialize daily shifts list
    daily_shifts = []
    
    # Loop over each day in the year
    for date in shifts.index:
        # List of available employees for today (those who haven't reached 40 hours/week)
        available_employees = [employee for employee in employees if sum(work_schedule[employee]) < working_hours_per_week]
        
        # Ensure that at least 75% of employees work
        if len(available_employees) < min_employees_working:
            # If not enough available employees, shuffle to meet the requirement
            eligible_employees = np.random.choice(employees, min_employees_working, replace=False)
        else:
            # Otherwise, pick randomly from available employees
            eligible_employees = np.random.choice(available_employees, min_employees_working, replace=False)
        
        # Assign shifts (8 hours per day) to selected employees
        daily_shifts.append(eligible_employees.tolist())
        
        # Update work schedule and rest days
        for employee in eligible_employees:
            # Ensure the employee doesn't exceed the weekly limit (no more than 40 hours per week)
            work_schedule[employee].append(hours_per_day)
            
            # Track rest days to ensure no more than 3 consecutive rest days
            rest_schedule[employee].append(False)  # Employee is working today (not resting)
            
            # Reset rest days for the employee if they worked today
            if len(rest_schedule[employee]) > 1 and rest_schedule[employee][-2] == False:
                rest_schedule[employee] = rest_schedule[employee][-1:]
    
    return daily_shifts

# Assign shifts for the whole year
daily_shifts = assign_shifts()

# Add the daily shifts to the DataFrame
shifts['Shifts'] = daily_shifts

# Now create a cross-tabulated table where each day is a row and each employee is a column
work_table = pd.DataFrame(index=shifts.index, columns=employees)

# Fill the table with 'X' if the employee is working on that day
for date, shift_list in zip(shifts.index, shifts['Shifts']):
    work_table.loc[date, shift_list] = 'X'  # Mark 'X' for employees working on that day
    # Fill NaN (empty days) with empty strings
    work_table.fillna('', inplace=True)

# Display the first 10 rows of the cross-tabulated table
work_table



,Employee_1,Employee_2,Employee_3,Employee_4,Employee_5,Employee_6,Employee_7,Employee_8,Employee_9,Employee_10,...,Employee_141,Employee_142,Employee_143,Employee_144,Employee_145,Employee_146,Employee_147,Employee_148,Employee_149,Employee_150
2025-01-01,X,,X,,X,X,X,X,X,,...,X,X,X,X,X,X,X,X,X,X
2025-01-02,X,X,X,X,,,,X,X,X,...,X,,,X,X,X,,X,X,X
2025-01-03,,X,X,X,,X,X,X,X,,...,X,,X,X,X,X,X,X,X,X
2025-01-04,X,,X,X,,,X,X,X,X,...,X,,X,X,,X,X,,X,
2025-01-05,X,X,X,,X,X,X,X,X,X,...,,,X,X,X,X,X,X,X,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,X,X,X,X,X,,X,X,X,X,...,X,,X,X,,X,X,X,X,
2025-12-28,X,X,X,X,X,X,X,X,X,X,...,X,X,,X,X,,X,X,X,X
2025-12-29,X,X,X,X,X,X,,X,X,X,...,X,X,X,X,X,,,,X,
2025-12-30,,X,X,,X,X,,X,,X,...,X,X,X,X,X,X,,X,X,X


In [ ]:
work_table["Employee_1"]

In [ ]:
output_filename = "employee_schedule_2025.xlsx"
work_table.to_excel(output_filename, sheet_name="Work Schedule")